## Setup

In [1]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random

## Input data directory
data_dir = "cureus"
inputdirectory = Path(f"./data_input/{data_dir}")
## This is where the output csv files will be written
out_dir = data_dir
outputdirectory = Path(f"./data_output/{out_dir}")

## Load Documents

In [2]:
## Dir PDF Loader
# loader = PyPDFDirectoryLoader(inputdirectory)
## File Loader
# loader = PyPDFLoader("C:\\Users\\go2za\\Documents\\knowledge_graph\\data_input\\cureus\\C2024C00104.pdf")
# loader = DirectoryLoader(inputdirectory, show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
print(pages[3].page_content)


Number of chunks =  398
Renewable Energy (Electricity) Act 2000 i
Compilation No. 30 Compilation date: 20/03/2024 Registered: 02/04/2024Contents
Part 1—Preliminary 1
1 Short title............................................................................................. 1
2 Commencement .................................................................................. 1
3 Object/outline...................................................................................... 1
4 Years to which this Act applies .......................................................... 2
5 Definitions .......................................................................................... 2
6 Act binds Crown ............................................................................... 12
7 Application to the external Territories.............................................. 12
7A Tax deductibility............................................................................... 12
7B Constitutional basis

In [2]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

# Function to load text files from a directory
def load_txt_documents(directory_path):
    documents = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory_path, filename)
            loader = TextLoader(file_path, encoding='ISO-8859-1')
            documents.extend(loader.load())
    return documents

# Load text documents
directory_path = "C:\\Users\\go2za\\Downloads\\AliAustralia\\Stakeholder"
documents = load_txt_documents(directory_path)

# Split the loaded documents into chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)

# Output the number of chunks and display a specific chunk
print("Number of chunks = ", len(pages))
print(pages[0].page_content)


Number of chunks =  31
This $30 billion project plans to power Singapore with Australian solar - create digital


In [9]:
type(txt_files_contents)

dict

## Create a dataframe of all the chunks

In [6]:
pip install yachalk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from helpers.df_helpers import documents2Dataframe
df = documents2Dataframe(pages)
print(df.shape)
df.head()

(31, 3)


text  \
0  This $30 billion project plans to power Singap...   
1  The multi-gigawatt Australia-Asia PowerLink (A...   
2  harness Australiaâs abundant solar resources...   
3  âAny time you are doing something for the fi...   
4  Colin Watson MIEAust\n\n\n\nâThe physical si...   

                                              source  \
0  C:\Users\go2za\Downloads\AliAustralia\Stakehol...   
1  C:\Users\go2za\Downloads\AliAustralia\Stakehol...   
2  C:\Users\go2za\Downloads\AliAustralia\Stakehol...   
3  C:\Users\go2za\Downloads\AliAustralia\Stakehol...   
4  C:\Users\go2za\Downloads\AliAustralia\Stakehol...   

                           chunk_id  
0  c6511989c2c049ab9fe8a53e294bd601  
1  da5cade0de7f4a43aa1a1e05ae6eb9a4  
2  0d63f158b0494adf8cc02cf1363a40c9  
3  dafddc7a30104d1ab3373f001722b21c  
4  0207d441410f4ebf8281b4fde9378f84

## Extract Concepts

In [7]:
## This function uses the helpers/prompt function to extract concepts from text
from helpers.df_helpers import df2Graph
from helpers.df_helpers import graph2Df

If regenerate is set to True then the dataframes are regenerated and Both the dataframes are written in the csv format so we dont have to calculate them again. 

        dfne = dataframe of edges

        df = dataframe of chunks


Else the dataframes are read from the output directory

In [8]:
# To regenerate the graph with LLM, set this to True
regenerate = True

if regenerate:
    concepts_list = df2Graph(df, model='zephyr:latest')
    dfg1 = graph2Df(concepts_list)
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)
    
    dfg1.to_csv(outputdirectory/"graph.csv", sep="|", index=False)
    df.to_csv(outputdirectory/"chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv(outputdirectory/"graph.csv", sep="|")

dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 

# Increasing the weight of the relation to 4. 
# We will assign the weight of 1 when later the contextual proximity will be calculated.  
print(dfg1.shape)
dfg1.head()

[
   {
       "node_1": "$30 billion",
       "node_2": "project",
       "edge": "The $30 billion project is a planned endeavor."
   },
   {
       "node_1": "Singapore",
       "node_2": "power",
       "edge": "The $30 billion project plans to provide power to Singapore."
   },
   {
       "node_1": "Australian solar",
       "node_2": "project",
       "edge": "The $30 billion project involves using solar energy from Australia."
   },
   {
       "node_1": "digital",
       "node_2": "create",
       "edge": "This $30 billion project also aims to generate digital capabilities."
   }
][
   {
       "node_1": "Australia-Asia PowerLink (AAPowerLink)",
       "node_2": "renewable electricity",
       "edge": "The multi-gigawatt Australia-Asia PowerLink (AAPowerLink) is a high-capacity solar generation, storage, and transmission system that will send large volumes of renewable electricity from the Northern Territory to markets in Darwin and Singapore."
   }, {
       "node_1": "Australi

[
   {
      "node_1": "project",
      "node_2": "future",
      "edge": "The speaker expresses enthusiasm about the future of the project, implying a positive relationship between the two concepts."
   },
   {
      "node_1": "first time",
      "node_2": "project",
      "edge": "The project is being undertaken for the first time, indicating a relation between the two terms."
   },
   {
      "node_1": "project",
      "node_2": "challenges",
      "edge": "The speaker acknowledges that challenges will arise during the course of the project, implying a relationship between the two concepts."
   }
][
  {
    "node_1": "Sun Cable",
    "node_2": "International nature of the project",
    "edge": "The international nature of the project presents unique challenges for Sun Cable, including getting approval from Indonesia whose waters the cable will need to run through in order to reach Singapore."
  },
  {
    "node_1": "Sun Cable",
    "node_2": "Successful design and delivery of the pr

   }
][
   {
       "node_1": "nuclear energy plant",
       "node_2": "Nuclear Activities (Prohibition) Act 1983 (Vic)",
       "edge": "In Victoria, the construction or operation of a nuclear energy plant is prevented by the Nuclear Activities (Prohibition) Act 1983."
   },
   {
       "node_1": "nuclear energy plant",
       "node_2": "Nuclear Facilities Prohibition Act 2000 (Qld)",
       "edge": "In Queensland, the construction or operation of a nuclear energy plant is prevented by the Nuclear Facilities Prohibition Act 2000."
   },
   {
       "node_1": "nuclear energy plant",
       "node_2": "Uranium Mining and Nuclear Facilities (Prohibitions) Act 1986 (NSW)",
       "edge": "In New South Wales, the construction or operation of a nuclear energy plant is prevented by the Uranium Mining and Nuclear Facilities (Prohibitions) Act 1986."
   },
   {
       "node_1": "nuclear fuel or waste",
       "node_2": "South Australia",
       "edge": "In South Australia, the conversion and en

       "node_1": "civil nuclear security and safeguards",
       "node_2": "site licencing",
       "node_3": "construction of new reactors",
       "node_4": "operation of nuclear energy plants",
       "node_5": "decommissioning, fuel and waste",
       "node_6": "storage and transport of radioactive materials",
       "node_7": "research",
       "edge": "",
       "type": "and"
   },
   {
       "node_1": "guidance published by the International Atomic Energy Agency",
       "node_2": "relevant for this",
       "edge": "",
       "type": "or"
   }
][
   {
       "node_1": "nuclear energy",
       "node_2": "social licence",
       "edge": "Nuclear energy plants require ongoing broad social acceptance including for the entire life cycle of such reactors due to social licence considerations. This is indicated by Senate inquiries and work undertaken by the Lowy Institute which suggests that a large part of the Australian population is unsure or unconvinced of the need for Australia t

       "edge": "The author suggests amending approvals regimes in line with the global trend, which may involve collaboration between the EPBC Act and the states and territories."
   }, {
       "node_1": "Indigenous Australians",
       "node_2": "energy optionality and mix",
       "edge": "The author mentions that perspectives of Indigenous Australians on energy optionality and mix need to be considered."
   }, {
       "node_1": "bipartisanship",
       "node_2": "loss of social cohesion",
       "edge": "The author highlights the importance of avoiding loss of social cohesion by ensuring bipartisanship."
   }, {
       "node_1": "waste recycling and management",
       "node_2": "intergenerational management of waste disposal",
       "edge": "The author mentions that waste recycling and management should include intergenerational management of waste disposal."
   }, {
       "node_1": "health and safety",
       "node_2": "workforce capability and potential for capacity building"

       "edge": "A communication strategy dedicated exclusively to the Agenda 2030 in France is currently being discussed with the main players committed to the SDGs."
   },
   {
       "node_1": "Communication strategy",
       "node_2": "Raise awareness",
       "edge": "The priority of the communication strategy dedicated exclusively to the Agenda 2030 in France is to raise awareness of the SDGs among as many people as possible."
   },
   {
       "node_1": "Communication strategy",
       "node_2": "National events",
       "edge": "The communication strategy dedicated exclusively to the Agenda 2030 in France aims to raise awareness of the SDGs among as many people as possible by taking part in national events."
   },
   {
       "node_1": "Newsletter",
       "node_2": "SDGs",
       "edge": "A monthly newsletter, 'ODDyssée vers 2030', aims to share news about the Sustainable Development Goals around the world, in Europe and in France."
   },
   {
       "node_1": "Newsletter",
   

       "node_1": "SDGs",
       "node_2": "Local actions",
       "edge": "Local actions can be highlighted using the SDGs as a frame of reference, making them more meaningful in the context of sustainable development."
   },
   {
       "node_1": "Non-state actors",
       "node_2": "Committed stakeholders",
       "edge": "Both non-state actors and committed stakeholders are involved in grassroots actions, projects, experiments, and educational initiatives related to the challenges of sustainable development."
   },
   {
       "node_1": "Non-state actors",
       "node_2": "Local stakeholders",
       "edge": "Non-state actors often work with local stakeholders such as associations and research institutions in their grassroots actions."
   },
   {
       "node_1": "Research",
       "node_2": "SDGs",
       "edge": "Research institutions are using the SDGs as an object of study to make their actions more innovative, responsible, and sustainable."
   }
][
   {
       "node_1": "Compa

       "node_1": "nuclear power",
       "node_2": "Singapore",
       "edge": "Singapore is interested in enhancing its knowledge and capabilities on nuclear power."
   },
   {
       "node_1": "decision",
       "node_2": "Singapore",
       "edge": "But itâs not so much about âletâs identify a nuclear projectâ, because we have not even decided to adopt nuclear power for Singapore."
   },
   {
       "node_1": "nuclear power",
       "node_2": "France",
       "edge": "France has expertise running nuclear power plants and is doing research in advanced nuclear technologies."
   },
   {
       "node_1": "percentage",
       "node_2": "France",
       "edge": "The country derives about 70 per cent of its electricity from nuclear energy with 56 operable reactors, according to the World Nuclear Association."
   },
   {
       "node_1": "Singapore",
       "node_2": "advanced nuclear technologies",
       "edge": "Singapore has been tracking advanced nuclear technologies and aims t

node_1                 node_2  \
0                             $30 billion                project   
1                               singapore                  power   
2                        australian solar                project   
3                                 digital                 create   
4  australia-asia powerlink (aapowerlink)  renewable electricity   

                                                edge  \
0     The $30 billion project is a planned endeavor.   
1  The $30 billion project plans to provide power...   
2  The $30 billion project involves using solar e...   
3  This $30 billion project also aims to generate...   
4  The multi-gigawatt Australia-Asia PowerLink (A...   

                           chunk_id node_3 node_4 node_5 node_6 type node_7  \
0  c6511989c2c049ab9fe8a53e294bd601    NaN    NaN    NaN    NaN  NaN    NaN   
1  c6511989c2c049ab9fe8a53e294bd601    NaN    NaN    NaN    NaN  NaN    NaN   
2  c6511989c2c049ab9fe8a53e294bd601    NaN    NaN    NaN    NaN  NaN    NaN   
3  c6511989c2c049ab9fe8a53e294bd601    NaN    NaN    NaN    NaN  NaN    NaN   
4  da5cade0de7f4a43aa1a1e05ae6eb9a4    NaN    NaN    NaN    NaN  NaN    NaN   

  edge_1 edge_2  count  
0    NaN    NaN      4  
1    NaN    NaN      4  
2    NaN    NaN      4  
3    NaN    NaN      4  
4    NaN    NaN      4

## Calculating contextual proximity

In [9]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

node_1  \
1565                                 water and sewerage   
1568                                 water and sewerage   
1569                                 water and sewerage   
1570                              whole of system costs   
1578  workforce capability and potential for capacit...   

                                                 node_2  \
1565                    planning consents and approvals   
1568                           state and territory laws   
1569  state and territory laws regulate the ownershi...   
1570                                      affordability   
1578                                           epbc act   

                                               chunk_id  count  \
1565  64e501a9d5464517bd30d36d6121ef4f,64e501a9d5464...      3   
1568  64e501a9d5464517bd30d36d6121ef4f,64e501a9d5464...      3   
1569  64e501a9d5464517bd30d36d6121ef4f,64e501a9d5464...      3   
1570  33b3e5b51aeb4f889f1f05e9538eacd0,33b3e5b51aeb4...      2   
1578  1d327ea57c0b4a378aae7e83b5462d96,1d327ea57c0b4...      2   

                      edge  
1565  contextual proximity  
1568  contextual proximity  
1569  contextual proximity  
1570  contextual proximity  
1578  contextual proximity

In [10]:
import os
import pandas as pd

# Define output directory
output_directory = "./saved_data"

# Create the directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Save DataFrames
dfg1.to_csv(os.path.join(output_directory, "stakeholder_dfg1.csv"), index=False)
dfg2.to_csv(os.path.join(output_directory, "stakeholder_dfg2.csv"), index=False)

print("DataFrames saved successfully.")


DataFrames saved successfully.


### Merge both the dataframes

In [ ]:
print(dfg1.shape)
print(dfg2.shape)
print(dfg1.columns)
print(dfg2.columns)
# dfg1 = dfg1.drop(columns=['node_3'])


In [11]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({
        "chunk_id": lambda x: ",".join(map(str, x)),
        "edge": lambda x: ','.join(map(str, x)),
        'count': 'sum'
    })
    .reset_index()
)



In [12]:
dfg.dropna

<bound method DataFrame.dropna of                                                 node_1  \
0                                          $30 billion   
1                            17-20 gigawatt solar farm   
2    4200 km of high voltage direct current (hvdc) ...   
3                        advanced nuclear technologies   
4                        advanced nuclear technologies   
..                                                 ...   
556                                 water and sewerage   
557                                 water and sewerage   
558                                 water and sewerage   
559                              whole of system costs   
560  workforce capability and potential for capacit...   

                                                node_2  \
0                                              project   
1               australia-asia powerlink (aapowerlink)   
2               australia-asia powerlink (aapowerlink)   
3                                    

In [38]:
import os
import pandas as pd

# Define output directory
output_directory = "./saved_data"

# Create the directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Save DataFrames
dfg.to_csv(os.path.join(output_directory, "stakeholder.csv"), index=False)

print("DataFrames saved successfully.")


DataFrames saved successfully.


In [13]:
# dfg = pd.concat([dfg1, dfg2], axis=0)
# dfg = (
#     dfg.groupby(["node_1", "node_2"])
#     .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
#     .reset_index()
# )
# dfg

## Calculate the NetworkX Graph

In [28]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(177,)

In [29]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

In [30]:
# Centrality Calculations
degree_centrality = nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G, normalized=True)

# Add centrality measures as node attributes
for node in G.nodes():
    G.nodes[node]['degree_centrality'] = degree_centrality[node]
    G.nodes[node]['closeness_centrality'] = closeness_centrality[node]
    G.nodes[node]['betweenness_centrality'] = betweenness_centrality[node]


### Calculate communities for coloring the nodes

In [31]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  14
[['$30 billion', '17-20 gigawatt solar farm', '4200 km of high voltage direct current (hvdc) submarine cable', 'australia-asia powerlink (aapowerlink)', 'australian economy', 'australian export industry', 'australian solar', 'carbon emissions abatement', 'challenges', 'colin watson mieaust', 'create', 'digital', 'first nations stakeholders', 'first time', 'future', 'indonesia', 'international nature of the project', 'northern territory chief minister michael gunner', 'power', 'project', 'renewable electricity', 'smec australia', 'successful design and delivery of the project', 'sun cable', 'tens of thousands of direct and indirect jobs'], ['a sufficiently compelling technological, economic and social licence case could be made in support of such plants', 'lift the commonwealth and state moratoria legislation as it applies to nuclear energy plants'], ['advanced nuclear technologies', 'cleaner form of nuclear power still under development', 'core inflation', '

### Create a dataframe for community colors

In [32]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

node    color  group
0                                          $30 billion  #bd57db      1
1                            17-20 gigawatt solar farm  #bd57db      1
2    4200 km of high voltage direct current (hvdc) ...  #bd57db      1
3               australia-asia powerlink (aapowerlink)  #bd57db      1
4                                   australian economy  #bd57db      1
..                                                 ...      ...    ...
172  the architecture of a broader framework might ...  #db5788     12
173                                       january 2025  #57d3db     13
174            singapore's platform worker protections  #57d3db     13
175  national radioactive waste management act 2012...  #db5f57     14
176                             state or territory law  #db5f57     14

[177 rows x 3 columns]

### Add colors to the graph

In [33]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [37]:
from pyvis.network import Network
# Create a Pyvis Network graph

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

# Add nodes with centrality info to Pyvis
for node in G.nodes():
    title = f"""
    Node: {node}<br>
    Degree Centrality: {G.nodes[node]['degree_centrality']:.4f}<br>
    Closeness Centrality: {G.nodes[node]['closeness_centrality']:.4f}<br>
    Betweenness Centrality: {G.nodes[node]['betweenness_centrality']:.4f}
    """
    net.add_node(
        node, 
        label=str(node),
        title=title,
        group=G.nodes[node]['group'],
        color=G.nodes[node]['color'],
        size=G.nodes[node]['betweenness_centrality'] * 100  # Scale size by betweenness centrality
    )

# Add edges
for index, row in dfg.iterrows():
    net.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

# Save the network as HTML
graph_output_directory = "./docs/stakeholder_index.html"

def save_network_with_utf8(net, path):
    html_content = net.generate_html()
    with open(path, "w", encoding="utf-8") as f:
        f.write(html_content)

net.show_buttons(filter_=["physics"])
save_network_with_utf8(net, graph_output_directory)
print(f"Graph saved to {graph_output_directory}")

Graph saved to ./docs/stakeholder_index.html


In [24]:
pip install pyvis


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   - ------------------------------------- 30.7/756.0 kB 660.6 kB/s eta 0:00:02
   - ------------------------------------- 30.7/756.0 kB 660.6 kB/s eta 0:00:02
   --- ----------------------------------- 61.4/756.0 kB 409.6 kB/s eta 0:00:02
   ----- -------------------------------- 112.6/756.0 kB 656.4 kB/s eta 0:00:01
   ------- ------------------------------ 153.6/756.0 kB 654.6 kB/s eta 0:00:01
   ----------- -------------------------- 225.3/756.0 kB 808.4 kB/s eta 0:00:01
   -------------- ----------------------- 286.7/756.0 kB 886.2 kB/s eta 0:00:01
   --------------- ---------------------- 317.4/756.0 kB 893.0 kB/s eta 0:00:01
   ------------------ ------------------- 368.6/756.0 kB 916.6 kB/s eta 0:00:01
   --------------------- ---------------- 419.8/756.0 kB 937.3 kB/s eta 0:00:01
   ------------------------ ------------- 481.3/756.0 kB 942.1 kB/s eta 0:00:01
   -------------------------- ----------- 522.2/